In [1]:
from langchain_google_genai  import ChatGoogleGenerativeAI

from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
class GemmaRouter:
    """
    A simple wrapper for using Google's Gemma-2-2B-IT model
    via Hugging Face's OpenAI-compatible inference API.
    """

    def __init__(
        self,
        model: str = "google/gemma-2-2b-it:nebius",
        token_env: str = "HF_THIRD_TOKEN",
        base_url: str = "https://router.huggingface.co/v1",
    ):
        load_dotenv()
        api_key = os.getenv(token_env)
        if not api_key:
            raise ValueError(
                f"Missing token: environment variable '{token_env}' not found."
            )

        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model

    def invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Non-streaming response."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.choices[0].message.content.strip()

    def stream_invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Streaming response."""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
            stream=True,
        )
        return stream
model2=GemmaRouter()


In [3]:
from langchain_core.messages import HumanMessage
messageToChatbot=[HumanMessage(content="My favourite color is blue")]

In [4]:
model2.invoke(messageToChatbot[0].content)

"That's awesome! Blue is a beautiful and calming color.  \n\nDo you have a favorite shade of blue? 😊 💙"

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
chatbotMemory={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id]=ChatMessageHistory()
    return chatbotMemory[session_id]

chatbot_with_message_history=RunnableWithMessageHistory(model2,get_session_history=get_session_history)